# Credit Card Fraud Detection

In [1]:
#importing necessary dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
#load dataset
data=pd.read_csv('creditcard.csv')
data.head(2)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0


In [3]:
#information about data
data.info()
data['Class'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

0    284315
1       492
Name: Class, dtype: int64

In [4]:
#since 0 represents a normal transaction and 1 represents a fraudulent transcation, classifying the data for analysis 
valid=data[data.Class==0]
fraud=data[data.Class==1]
print(valid.shape,fraud.shape)


(284315, 31) (492, 31)


In [5]:
valid.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [6]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

Taking a sample dataset containing similar distribution of normal/valid transactions and Fraudulent transaction

In [7]:
valid_sample=valid.sample(n=492)

In [8]:
#concatenating two dataframes
final_data=pd.concat([valid_sample,fraud], axis=0)
final_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
135506,81283.0,1.380400,-1.138373,0.195816,-1.544399,-1.106541,-0.060193,-0.965891,0.004414,-2.200620,...,-0.123554,-0.045423,-0.037140,-0.305265,0.346431,-0.174284,0.033084,0.011139,55.74,0
99983,67369.0,-1.819781,1.592793,-1.651890,-1.101122,1.676461,3.027206,-0.883303,2.111151,-0.794049,...,0.134339,-0.035004,-0.083094,1.036266,-0.019429,0.319246,-0.039394,0.031714,6.27,0
208173,137001.0,-5.722709,0.837407,-2.352195,-2.610065,1.168136,-0.547367,1.699807,-1.284554,0.821494,...,-0.138548,1.142934,-1.286403,0.014451,1.803330,0.112595,-3.119096,1.098372,30.00,0
112331,72614.0,-0.665353,0.474942,1.274875,-0.113750,0.279357,-0.364606,0.990518,0.033517,-1.032260,...,-0.201520,-0.873832,0.486268,0.182989,-0.888932,-0.141950,0.070464,0.159113,99.90,0
78510,57563.0,1.178063,0.133173,0.385851,1.400029,-0.260862,-0.215011,-0.033714,0.063893,0.497216,...,-0.121896,-0.163620,-0.039152,0.057336,0.620024,-0.307474,0.032404,0.011840,1.00,0


In [9]:
final_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,95175.467480,-0.029823,0.113570,-0.019021,0.085094,0.012882,-0.019497,0.025967,0.020231,-0.041229,...,0.055958,0.003674,-0.015622,-0.016561,0.01502,-0.012819,0.014360,-0.013192,0.002452,93.527398
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.10513,0.041449,0.051648,0.170575,0.075667,122.211321


Splitting the dataset into features and targets 

In [10]:
X=final_data.drop(columns='Class',axis=1)
Y=final_data['Class']

In [11]:
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
135506   81283.0  1.380400 -1.138373  0.195816 -1.544399 -1.106541 -0.060193   
99983    67369.0 -1.819781  1.592793 -1.651890 -1.101122  1.676461  3.027206   
208173  137001.0 -5.722709  0.837407 -2.352195 -2.610065  1.168136 -0.547367   
112331   72614.0 -0.665353  0.474942  1.274875 -0.113750  0.279357 -0.364606   
78510    57563.0  1.178063  0.133173  0.385851  1.400029 -0.260862 -0.215011   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [12]:
print(Y)

135506    0
99983     0
208173    0
112331    0
78510     0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


Split the dataset into training and testing sets

In [13]:
X_train,X_test,Y_train,Y_test= train_test_split(X,Y, test_size=0.2,stratify=Y,random_state=2)

Create and train the logistic regression model

In [14]:
model=LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

Predict accuracy score 

In [15]:
X_train_prediction=model.predict(X_train)
training_accuracy=accuracy_score(X_train_prediction,Y_train)

In [16]:
print("The accuracy score on Training Data : ",training_accuracy)

The accuracy score on Training Data :  0.9263024142312579


In [17]:
X_test_prediction=model.predict(X_test)
test_accuracy=accuracy_score(X_test_prediction,Y_test)

In [18]:
print("The accuracy score on Test Data : ",test_accuracy)

The accuracy score on Test Data :  0.8629441624365483


In [1]:
import tensorflow

ModuleNotFoundError: No module named 'tensorflow'